# NOTE:  THIS NOTEBOOK WILL TAKE 5-10 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.


# Analyze Data Quality with SageMaker Processing Jobs and Spark

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Spark are used to process and analyze data sets in order to detect data quality issues and prepare them for model training.  

In this notebook we'll use Amazon SageMaker Processing with a library called [**Deequ**](https://github.com/awslabs/deequ), and leverage the power of Spark with a managed SageMaker Processing Job to run our data processing workloads.

Here are some great resources on Deequ: 
* Blog Post:  https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/
* Research Paper:  https://assets.amazon.science/4a/75/57047bd343fabc46ec14b34cdb3b/towards-automated-data-quality-management-for-machine-learning.pdf

![Deequ](./img/deequ.png)

![Processing Job](./img/processing.jpg)

# Amazon Customer Reviews Dataset

https://s3.amazonaws.com/dsoaws/amazon-reviews-pds/readme.html

### Dataset Columns:

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.

In [1]:
%store -r ingest_create_athena_table_tsv_passed

In [2]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE NOTEBOOKS IN THE INGEST FOLDER FIRST. You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [3]:
print(ingest_create_athena_table_tsv_passed)

True


In [4]:
if not ingest_create_athena_table_tsv_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE NOTEBOOKS IN THE INGEST FOLDER FIRST. You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


# Run the Analysis Job using a SageMaker Processing Job with Spark
Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built with our Spark script.

In [5]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# Review the Spark preprocessing script.

In [6]:
!pygmentize preprocess-deequ-pyspark.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-deps", "pydeequ==0.1.5"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas==1.1.4"])

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import *

from pydeequ.analyzers import *
from pydeequ.checks import *
from pydeequ.verification import *
from pydeequ.suggestions import *

# PySpark Deequ GitHub Repo:  https://github.com/awslabs/python-deequ


def main():
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))

    # Retrieve the args and replace 's3://' with 's3a://' (used by Spark)
    s3_input_data = args["s3_input_data"].replace("s3://", "s3a://")
    print(s3_input_data)
    s3_output_analyze_dat

In [7]:
from sagemaker.spark.processing import PySparkProcessor

processor = PySparkProcessor(
    base_job_name="spark-amazon-reviews-analyzer",
    role=role,
    framework_version="2.4",
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    max_runtime_in_seconds=300,
)

In [8]:
s3_input_data = "s3://{}/amazon-reviews-pds/tsv/".format(bucket)
print(s3_input_data)

s3://sagemaker-us-east-1-211125778552/amazon-reviews-pds/tsv/


In [9]:
!aws s3 ls $s3_input_data

2024-02-26 04:15:27   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2024-02-26 04:15:28   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz
2024-02-26 04:15:30   12134676 amazon_reviews_us_Gift_Card_v1_00.tsv.gz


## Setup Output Data

In [10]:
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = "amazon-reviews-spark-analyzer-{}".format(timestamp_prefix)
processing_job_name = "amazon-reviews-spark-analyzer-{}".format(timestamp_prefix)

print("Processing job name:  {}".format(processing_job_name))

Processing job name:  amazon-reviews-spark-analyzer-2024-02-27-05-23-06


In [11]:
s3_output_analyze_data = "s3://{}/{}/output".format(bucket, output_prefix)

print(s3_output_analyze_data)

s3://sagemaker-us-east-1-211125778552/amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output


## Start the Spark Processing Job

_Notes on Invoking from Lambda:_
* However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess.py` file to S3 and specify the everything include --py-files, etc.
* We would need to do the following before invoking the Lambda:
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
* Then reference the s3://<location> above in the --py-files, etc.
* See Lambda example code in this same project for more details.

_Notes on not using ProcessingInput and Output:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by ProcessingInput and ProcessingOutput (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes._"
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use ProcessingInput, the data will be copied to each node (which we don't want in this case since Spark already handles this)

In [12]:
from sagemaker.processing import ProcessingOutput

processor.run(
    submit_app="preprocess-deequ-pyspark.py",
    submit_jars=["deequ-1.0.3-rc2.jar"],
    arguments=[
        "s3_input_data",
        s3_input_data,
        "s3_output_analyze_data",
        s3_output_analyze_data,
    ],
    logs=True,
    wait=False,
)


Job Name:  spark-amazon-reviews-analyzer-2024-02-27-05-23-06-073
Inputs:  [{'InputName': 'jars', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-211125778552/spark-amazon-reviews-analyzer-2024-02-27-05-23-06-073/input/jars', 'LocalPath': '/opt/ml/processing/input/jars', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-211125778552/spark-amazon-reviews-analyzer-2024-02-27-05-23-06-073/input/code/preprocess-deequ-pyspark.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []


In [13]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, processing_job_name
        )
    )
)

In [14]:
from IPython.core.display import display, HTML

processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After a Few Minutes</b>'.format(
            region, processing_job_name
        )
    )
)

In [15]:
from IPython.core.display import display, HTML

s3_job_output_prefix = output_prefix

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(
            bucket, s3_job_output_prefix, region
        )
    )
)

# Monitor the Processing Job

In [16]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

{'ProcessingInputs': [{'InputName': 'jars', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-211125778552/spark-amazon-reviews-analyzer-2024-02-27-05-23-06-073/input/jars', 'LocalPath': '/opt/ml/processing/input/jars', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-211125778552/spark-amazon-reviews-analyzer-2024-02-27-05-23-06-073/input/code/preprocess-deequ-pyspark.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingJobName': 'spark-amazon-reviews-analyzer-2024-02-27-05-23-06-073', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1, 'InstanceType': 'ml.m5.2xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 300}, 'AppSpecification':

In [17]:
running_processor.wait()

.............................02-27 05:27 smspark.cli  INFO     Parsing arguments. argv: ['/usr/local/bin/smspark-submit', '--jars', '/opt/ml/processing/input/jars', '/opt/ml/processing/input/code/preprocess-deequ-pyspark.py', 's3_input_data', 's3://sagemaker-us-east-1-211125778552/amazon-reviews-pds/tsv/', 's3_output_analyze_data', 's3://sagemaker-us-east-1-211125778552/amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output']
02-27 05:27 smspark.cli  INFO     Raw spark options before processing: {'jars': '/opt/ml/processing/input/jars', 'class_': None, 'py_files': None, 'files': None, 'verbose': False}
02-27 05:27 smspark.cli  INFO     App and app arguments: ['/opt/ml/processing/input/code/preprocess-deequ-pyspark.py', 's3_input_data', 's3://sagemaker-us-east-1-211125778552/amazon-reviews-pds/tsv/', 's3_output_analyze_data', 's3://sagemaker-us-east-1-211125778552/amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output']
02-27 05:27 smspark.cli  INFO     Rendered spark options: {'jar

Job ended with status 'Stopped' rather than 'Completed'. This could mean the job timed out or stopped early for some other reason: Consider checking whether it completed as you expect.


[/var/log/yarn/userlogs/application_1709011668638_0001/container_1709011668638_0001_01_000002/stderr] 24/02/27 05:28:50 INFO spark.MapOutputTrackerWorker: Don't have map outp


# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output 

## These are the quality checks on our dataset.

## _The next cells will not work properly until the job completes above._

In [18]:
!aws s3 ls --recursive $s3_output_analyze_data/

2024-02-27 05:28:36          0 amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/constraint-checks/_SUCCESS
2024-02-27 05:28:36        773 amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/constraint-checks/part-00000-0b562513-4c71-4d41-88d8-109a3901438e-c000.csv
2024-02-27 05:28:52          0 amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/constraint-suggestions/_SUCCESS
2024-02-27 05:28:52       8615 amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/constraint-suggestions/part-00000-942b0f4d-f598-434d-9ed2-8700d47a0b1b-c000.csv
2024-02-27 05:28:28          0 amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/dataset-metrics/_SUCCESS
2024-02-27 05:28:28        364 amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/dataset-metrics/part-00000-a7d4cb45-25f6-4fe8-a07d-71761211341e-c000.csv
2024-02-27 05:28:38          0 amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/success-metrics/_SUCCESS
2024-02-27 05:28:38        277 amazon-reviews-s

## Copy the Output from S3 to Local
* dataset-metrics/
* constraint-checks/
* success-metrics/
* constraint-suggestions/


In [19]:
!aws s3 cp --recursive $s3_output_analyze_data ./amazon-reviews-spark-analyzer/ --exclude="*" --include="*.csv"

download: s3://sagemaker-us-east-1-211125778552/amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/constraint-checks/part-00000-0b562513-4c71-4d41-88d8-109a3901438e-c000.csv to amazon-reviews-spark-analyzer/constraint-checks/part-00000-0b562513-4c71-4d41-88d8-109a3901438e-c000.csv
download: s3://sagemaker-us-east-1-211125778552/amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/constraint-suggestions/part-00000-942b0f4d-f598-434d-9ed2-8700d47a0b1b-c000.csv to amazon-reviews-spark-analyzer/constraint-suggestions/part-00000-942b0f4d-f598-434d-9ed2-8700d47a0b1b-c000.csv
download: s3://sagemaker-us-east-1-211125778552/amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/success-metrics/part-00000-a73ab2b2-7efa-4933-ae4d-a190ce14e2ec-c000.csv to amazon-reviews-spark-analyzer/success-metrics/part-00000-a73ab2b2-7efa-4933-ae4d-a190ce14e2ec-c000.csv
download: s3://sagemaker-us-east-1-211125778552/amazon-reviews-spark-analyzer-2024-02-27-05-23-06/output/dataset-metrics/part-0000

## Analyze Constraint Checks

In [20]:
import glob
import pandas as pd
import os


def load_dataset(path, sep, header):
    data = pd.concat(
        [pd.read_csv(f, sep=sep, header=header) for f in glob.glob("{}/*.csv".format(path))], ignore_index=True
    )

    return data

In [21]:
df_constraint_checks = load_dataset(path="./amazon-reviews-spark-analyzer/constraint-checks/", sep="\t", header=0)
df_constraint_checks[["check", "constraint", "constraint_status", "constraint_message"]]

,check,constraint,constraint_status,constraint_message
0,Review Check,SizeConstraint(Size(None)),Success,NaN
1,Review Check,"MinimumConstraint(Minimum(star_rating,None))",Success,NaN
2,Review Check,"MaximumConstraint(Maximum(star_rating,None))",Success,NaN
3,Review Check,"CompletenessConstraint(Completeness(review_id,...",Success,NaN
4,Review Check,UniquenessConstraint(Uniqueness(List(review_id...,Success,NaN
5,Review Check,CompletenessConstraint(Completeness(marketplac...,Success,NaN
6,Review Check,ComplianceConstraint(Compliance(marketplace co...,Success,NaN


## Analyze Dataset Metrics

In [22]:
df_dataset_metrics = load_dataset(path="./amazon-reviews-spark-analyzer/dataset-metrics/", sep="\t", header=0)
df_dataset_metrics

,entity,instance,name,value
0,Column,review_id,Completeness,1.000000
1,Column,review_id,ApproxCountDistinct,381704.000000
2,Mutlicolumn,"total_votes,star_rating",Correlation,-0.086052
3,Dataset,*,Size,396601.000000
4,Column,star_rating,Mean,4.102493
5,Column,top star_rating,Compliance,0.765893
6,Mutlicolumn,"total_votes,helpful_votes",Correlation,0.985751


## Analyze Success Metrics

In [23]:
df_success_metrics = load_dataset(path="./amazon-reviews-spark-analyzer/success-metrics/", sep="\t", header=0)
df_success_metrics

,entity,instance,name,value
0,Column,review_id,Completeness,1.0
1,Column,review_id,Uniqueness,1.0
2,Dataset,*,Size,396601.0
3,Column,star_rating,Maximum,5.0
4,Column,star_rating,Minimum,1.0
5,Column,"marketplace contained in US,UK,DE,JP,FR",Compliance,1.0
6,Column,marketplace,Completeness,1.0


## Analyze Constraint Suggestions

In [24]:
# df_constraint_suggestions = load_dataset(path='./amazon-reviews-spark-analyzer/constraint-suggestions/', sep='\t', header=0)

# pd.set_option('max_colwidth', 999)

# df_constraint_suggestions = df_constraint_suggestions[['_1', '_2', '_3']].dropna()
# df_constraint_suggestions.columns=['column_name', 'description', 'code']
# df_constraint_suggestions

# Release Resources

In [25]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [26]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>